In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, get_scheduler
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn

In [2]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m")
dataset = load_dataset("stanfordnlp/SHP")
# dataset = dataset.train_test_split(test_size=0.8)
system_prompt = tokenizer.bos_token + "system\n The following is a conversation between user and an AI assistant. " \
                                      "The assistant is helpful, creative, clever, and very friendly.\n" \
                + tokenizer.eos_token

Using custom data configuration stanfordnlp--SHP-c26141d67734c662


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /Users/sachin/.cache/huggingface/datasets/stanfordnlp___json/stanfordnlp--SHP-c26141d67734c662/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
def tokenize_function(example):
    dictionary = {}
    completion_a = tokenizer(system_prompt + tokenizer.bos_token + "user: " + example["history"] + tokenizer.eos_token +
                           tokenizer.bos_token + "assistant: " + example["human_ref_A"] + tokenizer.eos_token,
                           truncation=True)
    dictionary["input_ids_A"] = completion_a.pop("input_ids")
    dictionary["attention_mask_A"] = completion_a.pop("attention_mask")
    completion_b = tokenizer(system_prompt + tokenizer.bos_token + "user: " + example["history"] + tokenizer.eos_token +
                           tokenizer.bos_token + "assistant: " + example["human_ref_B"] + tokenizer.eos_token,
                           truncation=True)
    dictionary["input_ids_B"] = completion_b.pop("input_ids")
    dictionary["attention_mask_B"] = completion_b.pop("attention_mask")
    return dictionary

In [4]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000))
tokenized_dataset = train_dataset.map(tokenize_function)
tokenized_dataset = tokenized_dataset.remove_columns(['post_id', 'domain', 'upvote_ratio', 'history', 'c_root_id_A',
                                                      'c_root_id_B', 'created_at_utc_A', 'created_at_utc_B',
                                                      'human_ref_A', 'human_ref_B', 'labels', 'seconds_difference',
                                                      'score_ratio'])
tokenized_dataset.set_format("torch")
train_dataloader = DataLoader(tokenized_dataset, batch_size=1)

Parameter 'indices'=range(0, 10000) of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/10000 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
class RewardModel(nn.Module):
    def __init__(self):
        super(RewardModel, self).__init__()
        self.base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m")
        self.linear = nn.Linear(self.base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        base_model_output = self.base_model(input_ids, attention_mask, output_hidden_states=True)
        #We take mean of reward score for every token's last layer hidden state.
        #Another valid startegy can be to only consider reward score for the last token's last layer hidden state.
        return torch.mean(self.linear(base_model_output.hidden_states[-1].detach()))

In [6]:
reward_model = RewardModel()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
reward_model.to(device)
optimizer = AdamW(reward_model.linear.parameters(), lr=1e-5)
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [7]:
reward_model.train()
i = 0
running_loss = 0.0
training_loss = []
model_save_path = 'rm-pythia.pt'

In [ ]:
for batch in train_dataloader:
    i += 1
    print(i)
    batch = {k: v.to(device) for k, v in batch.items()}
    model_score_A = reward_model(input_ids=batch["input_ids_A"], attention_mask=batch["attention_mask_A"])
    model_score_B = reward_model(input_ids=batch["input_ids_B"], attention_mask=batch["attention_mask_B"])
    if batch["score_A"] > batch["score_B"]:
        loss = -1 * nn.functional.logsigmoid(model_score_A - model_score_B)
    else:
        loss = -1 * nn.functional.logsigmoid(model_score_B - model_score_A)
    loss.backward()
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    running_loss += loss.item()
    if i % 50 == 0:
        print(running_loss / 50)
        training_loss.append(running_loss / 50)
        running_loss = 0.0
        torch.save(reward_model.linear.state_dict(), model_save_path)
        i = 0
torch.save(reward_model.linear.state_dict(), model_save_path)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
0.7143135058879853
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
0.7218931090831756
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
0.6825545197725296
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
